In [21]:
#import
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.contingency_tables import mcnemar
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from scipy.special import comb
from statsmodels.stats.contingency_tables import mcnemar
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer
from itertools import combinations 
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from sklearn.model_selection import train_test_split, cross_val_score
import random

In [ ]:
#load data
#scale +preprocess
#models before 
#shuffling
#models 

In [10]:
#load dataframe with classifications

#this is just an example of what you should output AFTER classification using both shuffling methods + model types 
df = pd.DataFrame(columns=['trial','X', 'X2', 'Movement', 'Y', 'Classification_US_l', 'Classification_US_nl', 'Classification_S_l','Classification_S_nl' ])

#this is juts sample data 
for i in range(50):
    df.loc[i]=[str(i)] +[np.random.randint(0,2) for n in range(8)]

df.head()

,trial,X,X2,Movement,Y,Classification_US_l,Classification_US_nl,Classification_S_l,Classification_S_nl
0,0,0,1,1,0,0,1,0,1
1,1,0,0,1,1,0,0,1,1
2,2,0,0,1,0,0,0,1,0
3,3,1,1,0,0,0,1,0,1
4,4,1,1,1,1,0,1,0,1


In [11]:
#were the classifications correct?
df.Classification_US_l= (df.Classification_US_l.values==df.Y.values).astype(int)
df.Classification_S_l= (df.Classification_S_l.values==df.Y.values).astype(int)
df.Classification_US_nl= (df.Classification_US_nl.values==df.Y.values).astype(int)
df.Classification_S_nl= (df.Classification_S_nl.values==df.Y.values).astype(int)

In [12]:
print(confusion_matrix(df.Y.values.astype(int), df.Classification_US_l.values.astype(int)))
print(confusion_matrix(df.Y.values.astype(int), df.Classification_S_l.values.astype(int)))
print(confusion_matrix(df.Y.values.astype(int), df.Classification_US_nl.values.astype(int)))
print(confusion_matrix(df.Y.values.astype(int), df.Classification_S_nl.values.astype(int)))

[[ 8 16]
 [16 10]]
[[14 10]
 [15 11]]
[[17  7]
 [14 12]]
[[10 14]
 [13 13]]


In [20]:
#do bootstrapping 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_curve,roc_auc_score, precision_recall_curve, auc,plot_precision_recall_curve

#this is an example with log reg, put the SVM model in here once you refine it (Charlotte)

def BootstrapCoef(data,numboot):
    clf = LogisticRegression(penalty='none', max_iter=1000, solver='lbfgs')
    numboot = 1000
    n = len(data)
    accuracy = np.zeros((numboot, 1))   #5 
    for i in range(numboot):
        d = data.sample(n, replace=True)
        X_fit = np.c_[d.X.values.astype(int), d.X2.values.astype(int)]
        cver = StratifiedKFold(n_splits = 5)
        #accuracy[i,:] = cross_val_score(clf, X_fit,d.Y.astype(int), cv=cver, scoring = 'accuracy')
        accuracy[i] = np.mean(cross_val_score(clf, X_fit,d.Y.astype(int), cv=cver, scoring = 'accuracy'))
    return accuracy

#run this for each model/group
params = BootstrapCoef(df,100)
#params

array([[0.4 ],
       [0.82],
       [0.54],
       [0.64],
       [0.56],
       [0.58],
       [0.5 ],
       [0.58],
       [0.48],
       [0.6 ],
       [0.42],
       [0.38],
       [0.52],
       [0.58],
       [0.54],
       [0.56],
       [0.62],
       [0.56],
       [0.54],
       [0.4 ],
       [0.42],
       [0.6 ],
       [0.62],
       [0.64],
       [0.54],
       [0.62],
       [0.58],
       [0.56],
       [0.56],
       [0.62],
       [0.48],
       [0.62],
       [0.54],
       [0.62],
       [0.62],
       [0.64],
       [0.52],
       [0.6 ],
       [0.54],
       [0.56],
       [0.56],
       [0.78],
       [0.68],
       [0.6 ],
       [0.46],
       [0.32],
       [0.54],
       [0.64],
       [0.56],
       [0.54],
       [0.6 ],
       [0.6 ],
       [0.64],
       [0.5 ],
       [0.52],
       [0.52],
       [0.58],
       [0.66],
       [0.56],
       [0.54],
       [0.62],
       [0.54],
       [0.52],
       [0.56],
       [0.5 ],
       [0.42],
       [0.

In [22]:
#We need to create a dataframe like the example I have made below. To do this, instead of randomly assigning values
#fill the datframe with the bootstrapped data afer runing the model (Charlotte)

anova_df= pd.DataFrame(columns=['model', 'shuffling', 'movement', 'accuracy'])

model_levels= ['linear', 'nonlinear']
shuffling_levels= ['shuffled', 'unshuffled']
movement_levels= ['active', 'passive']
    
for j in range(50):
    anova_df.loc[j]=[random.choice(model_levels)]+[random.choice(shuffling_levels)]+[random.choice(movement_levels)]+ [random.uniform(0, 1)]
    

anova_df.head()                

,model,shuffling,movement,accuracy
0,linear,unshuffled,passive,0.165044
1,linear,unshuffled,active,0.301090
2,nonlinear,unshuffled,active,0.575812
3,linear,shuffled,active,0.695095
4,nonlinear,shuffled,passive,0.675106


In [23]:
#run the anova
anova_model = ols("accuracy ~ C(model, Sum) + C(movement, Sum) + C(shuffling, Sum) + C(model, Sum)*C(movement, Sum)*C(shuffling, Sum)", data=anova_df).fit()
aov_table = sm.stats.anova_lm(anova_model, typ=3)
aov_table

,sum_sq,df,F,PR(>F)
Intercept,11.797398,1.0,131.543679,1.604912e-14
"C(model, Sum)",0.049521,1.0,0.552175,4.615668e-01
"C(movement, Sum)",0.293503,1.0,3.272626,7.760650e-02
"C(shuffling, Sum)",0.009722,1.0,0.108404,7.436051e-01
"C(model, Sum):C(movement, Sum)",0.064424,1.0,0.718347,4.014907e-01
"C(model, Sum):C(shuffling, Sum)",0.012382,1.0,0.138060,7.120848e-01
"C(movement, Sum):C(shuffling, Sum)",0.042074,1.0,0.469136,4.971479e-01
"C(model, Sum):C(movement, Sum):C(shuffling, Sum)",0.009393,1.0,0.104734,7.478269e-01
Residual,3.766739,42.0,NaN,NaN


In [26]:
#do the posthoc test
# perform multiple pairwise comparison (Tukey HSD)
anova_df['combination'] = anova_df.movement + " / " + anova_df.model + " / " + anova_df.shuffling
m_comp = pairwise_tukeyhsd(endog=anova_df['accuracy'], groups=anova_df['combination'], alpha=0.05)


In [31]:
# coerce the tukeyhsd table to a DataFrame
tukey_data = pd.DataFrame(data=m_comp._results_table.data[1:], columns = m_comp._results_table.data[0])
tukey_data

,group1,group2,meandiff,p-adj,lower,upper,reject
0,active / linear / shuffled,active / linear / unshuffled,0.0977,0.9000,-0.3953,0.5908,False
1,active / linear / shuffled,active / nonlinear / shuffled,0.0140,0.9000,-0.5172,0.5453,False
2,active / linear / shuffled,active / nonlinear / unshuffled,0.1029,0.9000,-0.4483,0.6542,False
3,active / linear / shuffled,passive / linear / shuffled,-0.0552,0.9000,-0.7304,0.6200,False
4,active / linear / shuffled,passive / linear / unshuffled,-0.0240,0.9000,-0.5397,0.4917,False
5,active / linear / shuffled,passive / nonlinear / shuffled,-0.1376,0.9000,-0.6689,0.3936,False
6,active / linear / shuffled,passive / nonlinear / unshuffled,-0.2343,0.9000,-0.9095,0.4408,False
7,active / linear / unshuffled,active / nonlinear / shuffled,-0.0837,0.9000,-0.5543,0.3868,False
8,active / linear / unshuffled,active / nonlinear / unshuffled,0.0052,0.9000,-0.4879,0.4983,False
9,active / linear / unshuffled,passive / linear / shuffled,-0.1530,0.9000,-0.7815,0.4756,False
